### Exercise 6

In this excercise we have implemented three different marcov chain monte carlo sampling methods.

In [19]:
import numpy as np
from math import floor,factorial
from scipy import stats

In [20]:
floor(5.1)

5

## Part 1

In this part we have implemented one dimensional metropolis hasting. In that method we move a random step to get a new samplme using a symetrical distribution and the we check if the new sample has greater non normalized probability. Dipending on the result and some random coin flip we accept the new sample or we repeat the previous one. 

In [21]:
m=10
lambd = 1
s=8
A= lambd*s

In [22]:
def g(i):
    if (i > m): return 0
    if (i < 0): return 0
    return (A**i)/factorial(i)

We get a value from the $\chi^2$ test that is greater than 0.05. so it is an acceptale sample

## Part 2

### Question a

In [23]:
def sample_MCMC(sample_size):
    Xi=0
    sample = np.empty( 0 , dtype = 'float64' )
    sample = np.append( sample , Xi )
    for i in range(sample_size-1):
        Dxi= floor(10 * np.random.uniform(-1,1,1)[0])
        Yi = Xi + Dxi
        acceptance = 'reject'
        if (g(Yi)>=g(Xi)):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        if (np.random.uniform(0,1,1)[0]<=(g(Yi)/g(Xi))):
            sample = np.append( sample , Yi )
            Xi=Yi
            continue
        sample = np.append( sample , Xi )
    return sample

In [24]:
P = [g(i) for i in range(m)]
denominator = sum(P)
P = [i/denominator for i in (P) ]
X = sample_MCMC(10000)
sim_sum = sum(X)
Observ, bins = np.histogram(X, bins= m)
Observed = [i/sim_sum for i in Observ]
Expected = P
least_squares = [(Obs-Exp)**2/Exp for Obs, Exp in list(zip(Expected,Observed))]
chi_sqare_calculated_value = sum(least_squares)
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

0.8560111489829808

## Part 2

In this part we apply the same method at the two dimentions

### a 

In this part we in each step we move a the same time to two directions

In [25]:
def g_2d(x):
    i,j = x
    if (i >= m): return 0
    if (j >= m): return 0
    if (i+j > m): return 0
    if (i < 0): return 0
    if (j < 0): return 0
    if (i+j < 0): return 0
    return ((A**i)/factorial(i))*((A**j)/factorial(j))

In [26]:
def sample_MCMC_2d(sample_size):
    Xi=(0,0)
    sample = []
    sample.append( Xi )
    for i in range(sample_size-1):
        #print('sample=',sample)
        Dxi_1= floor(3* np.random.normal(0,1,1)[0])
        Dxi_2= floor(3* np.random.normal(0,1,1)[0])
        Yi = (Xi[1] + Dxi_1,Xi[1]+Dxi_2)
        acceptance = 'reject'
        #print('Xi=',Xi)
        #print('Yi=',Yi)
        #print('g(Xi)=',g(Xi))
        #print('g(Yi)=',g(Yi))
        #input()
        if (g_2d(Yi)>=g_2d(Xi)):
            sample.append( Yi )
            Xi=Yi
            continue
        if (np.random.uniform(0,1,1)[0]<=(g_2d(Yi)/g_2d(Xi))):
            sample.append(  Yi )
            Xi=Yi
            continue
        sample.append(  Xi )
    return sample

In [27]:
probs = np.zeros((m, m), dtype='float64')


In [28]:
for i in range(m):
    for j in range(i,m):
        probs[i,j] = g_2d((i,j))

In [29]:
P = probs
denominator = sum(P)
P = P/denominator

In [30]:
counts =  np.zeros((m, m), dtype='float64')
X = sample_MCMC_2d(10000)

for element in X:
    #print(element)
    i,j = element
    counts[i,j] += 1
sim_sum = sum(counts)
Observed = counts/sim_sum
Expected = P
least_squares = np.zeros((m, m), dtype='float64')
for i in range(m):
    for j in range(i,m):
        if (Expected[i,j] == 0.0): 
            least_squares[i,j]=0
            continue
        least_squares[i,j] = (Observed[i,j]-Expected[i,j])**2/Expected[i,j]
chi_sqare_calculated_value = sum(sum(least_squares))
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

0.9359768282466883

We get a value from the $\chi^2$ test that is greater than 0.05. so it is an acceptale sample

### Question b

In [36]:
def sample_MCMC_2d_dwise(sample_size):
    Xi=(0,0)
    sample = []
    sample.append( Xi )
    for i in range(sample_size-1):
        #print('sample=',sample)
        Dxi_1= floor(3* np.random.normal(0,1,1)[0])
        Dxi_2= floor(3* np.random.normal(0,1,1)[0])
        Yi_1d = (Xi[1] + Dxi_1,Xi[1])
        #print('Xi=',Xi)
        #print('Yi=',Yi)
        #print('g(Xi)=',g(Xi))
        #print('g(Yi)=',g(Yi))
        #input()
        if (g_2d(Yi_1d)>=g_2d(Xi)):
            Xi=Yi_1d
         
        elif (np.random.uniform(0,1,1)[0]<=(g_2d(Yi_1d)/g_2d(Xi))):
            Xi=Yi_1d
        Yi_2d = (Xi[1],Xi[1] + Dxi_2)
        
        
        if (g_2d(Yi_2d)>=g_2d(Xi)):
            sample.append(Yi_2d )
            Xi=Yi_2d
            continue
        if (np.random.uniform(0,1,1)[0]<=(g_2d(Yi_2d)/g_2d(Xi))):
            sample.append(  Yi_2d )
            Xi=Yi_2d
            continue
        
        sample.append(  Xi )
    return sample

In [37]:
counts =  np.zeros((m, m), dtype='float64')
X = sample_MCMC_2d_dwise(10000)

for element in X:
    #print(element)
    i,j = element
    counts[i,j] += 1
sim_sum = sum(counts)
Observed = counts/sim_sum
Expected = P
least_squares = np.zeros((m, m), dtype='float64')
for i in range(m):
    for j in range(i,m):
        if (Expected[i,j] == 0.0): 
            least_squares[i,j]=0
            continue
        least_squares[i,j] = (Observed[i,j]-Expected[i,j])**2/Expected[i,j]
chi_sqare_calculated_value = sum(sum(least_squares))
p_value = stats.chi2.sf(chi_sqare_calculated_value, m)
p_value

0.9274668395801717